In [ ]:
import rasterio
import numpy
import matplotlib.pyplot as plt
import os
import pandas as pd
from rasterio.plot import show
import matplotlib.colors as colors
from PIL import Image
from matplotlib import cm
import matplotlib.cm as cm
from sklearn import preprocessing

In [ ]:
src = rasterio.open('ndvi.tiff')

In [ ]:
band_red = src.read(3)
band_nir = src.read(1)

In [ ]:
numpy.seterr(divide='ignore', invalid='ignore')

nir = src.read(1)
red = src.read(3)

ndvi = (nir.astype(float) - red.astype(float)) / (nir.astype(float) + red.astype(float))

In [ ]:
ndvi.shape

In [ ]:
temp_array = ndvi.flatten()

In [ ]:
temp_array = temp_array[numpy.logical_not(numpy.isnan(temp_array))]

In [ ]:
normalized_arr = preprocessing.minmax_scale(temp_array, feature_range=(0, 1), axis=0, copy=True)
new_array1= list(normalized_arr)
dead_pixels = 12000000-len(new_array1)
for i in range(dead_pixels):
    new_array1.append(0)
normalized_arr = numpy.array(new_array1)

In [ ]:
yeni_ndvi=normalized_arr.reshape(3000, 4000)

In [ ]:
print(numpy.nanmin(yeni_ndvi))
print(numpy.nanmax(yeni_ndvi))

In [ ]:
from matplotlib import colors


class MidpointNormalize(colors.Normalize):
   
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
       
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return numpy.ma.masked_array(numpy.interp(value, x, y), numpy.isnan(value))

In [ ]:
min=numpy.nanmin(yeni_ndvi)
max=numpy.nanmax(yeni_ndvi)


mid=0.3


cmapp = cm.get_cmap('RdYlGn', 20) 
norm = MidpointNormalize(vmin=0, vmax=1, midpoint=mid)
fig = plt.figure(figsize=(10,10))


ax = fig.add_subplot(111)


cbar_plot = ax.imshow(ndvi, cmap=cmapp, vmin=0, vmax=1, norm=norm)



ax.axis('off')


ax.set_title('Normalized Difference Vegetation Index', fontsize=17, fontweight='bold')


cbar = fig.colorbar(cbar_plot, orientation='horizontal', shrink=0.65)


fig.savefig("25.tiff", dpi=200, bbox_inches='tight', pad_inches=0.7)


plt.show()